In [ ]:
import tensorflow as tf
import tensorflow.keras as K
import tensorflow_datasets as tfds
import pandas as pd

In [ ]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

In [ ]:
train_batches = train_data.shuffle(1000).padded_batch(10)
test_batches = test_data.shuffle(1000).padded_batch(10)

In [ ]:
encoder = info.features['text'].encoder
embedding_dim=16

model = K.Sequential([
  K.layers.Embedding(encoder.vocab_size, embedding_dim),
  K.layers.GlobalAveragePooling1D(),
  K.layers.Dense(16, activation='relu'),
  K.layers.Dense(1)
])

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    train_batches,
    epochs=10,
    validation_data=test_batches, validation_steps=20)

In [ ]:
!mkdir -p /tmp/word2vec/model
model.save('/tmp/word2vec/model')

!tar -cvzf /tmp/word2vec/model.tgz -C /tmp/word2vec/model .
!rm -fr /tmp/word2vec/model

embedding_weights = pd.DataFrame(
    model.layers[0].get_weights()[0][1:,:],
    index=pd.Index(
        [encoder.decode([i]).rstrip() for i in range(1, encoder.vocab_size)],
        name='term'))
embedding_weights.to_csv('/tmp/word2vec/embedding.csv')

!ls -l /tmp/word2vec